<a href="https://colab.research.google.com/github/waldals/MidTerm_Project/blob/main/datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_full = pd.read_pickle(r'/content/drive/MyDrive/df_full50k.pkl')

In [ ]:
df = df_full

In [ ]:
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
10778864,2018-02-09,F9,F9,F9,777,F9,N203FR,777,12889,LAS,"Las Vegas, NV",14831,SJC,"San Jose, CA",1257,1250.0,-7.0,18.0,1308.0,1413.0,3.0,1454,1438.0,-16.0,0.0,NaN,0.0,N,69.0,63.0,38.0,1.0,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12088984,2018-04-14,UA,UA_CODESHARE,UA,4043,EV,N14173,4043,11618,EWR,"Newark, NJ",10785,BTV,"Burlington, VT",1135,1125.0,-10.0,12.0,1137.0,1219.0,10.0,723,715.0,-8.0,0.0,NaN,0.0,N,83.0,81.0,48.0,1.0,246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14272108,2019-04-19,DL,DL,DL,747,DL,N904DE,747,10792,BUF,"Buffalo, NY",10397,ATL,"Atlanta, GA",1916,2156.0,160.0,10.0,2206.0,2400.0,6.0,2100,2104.0,4.0,0.0,NaN,0.0,N,190.0,192.0,176.0,1.0,1182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6164840,2019-11-09,AS,AS,AS,677,AS,N552AS,677,13871,OMA,"Omaha, NE",14747,SEA,"Seattle, WA",1815,1806.0,-9.0,11.0,1817.0,1948.0,4.0,1805,1757.0,-8.0,0.0,NaN,0.0,N,145.0,141.0,129.0,1.0,972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9767090,2019-01-24,NK,NK,NK,689,NK,N644NK,689,11433,DTW,"Detroit, MI",14635,RSW,"Fort Myers, FL",2015,2342.0,207.0,8.0,2350.0,220.0,13.0,2120,2053.0,-27.0,0.0,NaN,0.0,N,189.0,163.0,140.0,1.0,1017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.isnull().values.any()

True

In [ ]:
# missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
df2 = missing_data[missing_data['Percent'] > 0.4]
to_drop = df2.index.tolist()
df1 = df.drop(to_drop, axis=1, inplace=True)

In [ ]:
# missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
#replace NaNs with 0, or something 
df = df.replace(np.nan, 0)

In [ ]:
df.isnull().values.any()

False

In [ ]:
df = df.drop(['flights','mkt_unique_carrier','op_unique_carrier','origin_city_name','dest_airport_id','origin_airport_id','dest_city_name','dup','branded_code_share','tail_num','op_carrier_fl_num'],axis=1)

In [ ]:
df.head(1)

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance
10778864,2018-02-09,F9,777,LAS,SJC,1257,1250.0,-7.0,18.0,1308.0,1413.0,3.0,1454,1438.0,-16.0,0.0,0.0,69.0,63.0,38.0,216.0


In [ ]:
df.fl_date = pd.to_datetime(df.fl_date)
df['year'] = df.fl_date.dt.year
df['month'] = df.fl_date.dt.month
df['day'] = df.fl_date.dt.day

In [ ]:
df.to_pickle('/content/drive/MyDrive/df_clean_sample50k.pkl')

In [ ]:
df.to_csv('/content/drive/MyDrive/df_clean_sample50k.csv')

In [ ]:
df.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,year,month,day
10778864,2018-02-09,F9,777,LAS,SJC,1257,1250.0,-7.0,18.0,1308.0,1413.0,3.0,1454,1438.0,-16.0,0.0,0.0,69.0,63.0,38.0,216.0,2018,2,9
12088984,2018-04-14,UA,4043,EWR,BTV,1135,1125.0,-10.0,12.0,1137.0,1219.0,10.0,723,715.0,-8.0,0.0,0.0,83.0,81.0,48.0,246.0,2018,4,14
14272108,2019-04-19,DL,747,BUF,ATL,1916,2156.0,160.0,10.0,2206.0,2400.0,6.0,2100,2104.0,4.0,0.0,0.0,190.0,192.0,176.0,1182.0,2019,4,19
6164840,2019-11-09,AS,677,OMA,SEA,1815,1806.0,-9.0,11.0,1817.0,1948.0,4.0,1805,1757.0,-8.0,0.0,0.0,145.0,141.0,129.0,972.0,2019,11,9
9767090,2019-01-24,NK,689,DTW,RSW,2015,2342.0,207.0,8.0,2350.0,220.0,13.0,2120,2053.0,-27.0,0.0,0.0,189.0,163.0,140.0,1017.0,2019,1,24


In [ ]:
df_sub = df.drop(['mkt_carrier',	'mkt_carrier_fl_num',	'origin',	'dest'],axis=1)

In [ ]:
df_sub.head(1)

,fl_date,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,year,month,day
10778864,2018-02-09,1257,1250.0,-7.0,18.0,1308.0,1413.0,3.0,1454,1438.0,-16.0,0.0,0.0,69.0,63.0,38.0,216.0,2018,2,9


In [ ]:
df_dummy = pd.get_dummies(df)

In [ ]:
df_dummy.head()

,fl_date,mkt_carrier_fl_num,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,year,month,day,mkt_carrier_AA,mkt_carrier_AS,mkt_carrier_B6,mkt_carrier_DL,mkt_carrier_F9,mkt_carrier_G4,mkt_carrier_HA,mkt_carrier_NK,mkt_carrier_UA,mkt_carrier_VX,mkt_carrier_WN,origin_ABE,origin_ABI,origin_ABQ,origin_ABR,origin_ABY,origin_ACK,origin_ACT,origin_ACV,...,dest_SMX,dest_SNA,dest_SPI,dest_SPN,dest_SPS,dest_SRQ,dest_STC,dest_STL,dest_STS,dest_STT,dest_STX,dest_SUN,dest_SUX,dest_SWF,dest_SWO,dest_SYR,dest_TLH,dest_TOL,dest_TPA,dest_TRI,dest_TTN,dest_TUL,dest_TUS,dest_TVC,dest_TWF,dest_TXK,dest_TYR,dest_TYS,dest_UIN,dest_USA,dest_VEL,dest_VLD,dest_VPS,dest_WRG,dest_WYS,dest_XNA,dest_XWA,dest_YAK,dest_YKM,dest_YUM
10778864,2018-02-09,777,1257,1250.0,-7.0,18.0,1308.0,1413.0,3.0,1454,1438.0,-16.0,0.0,0.0,69.0,63.0,38.0,216.0,2018,2,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12088984,2018-04-14,4043,1135,1125.0,-10.0,12.0,1137.0,1219.0,10.0,723,715.0,-8.0,0.0,0.0,83.0,81.0,48.0,246.0,2018,4,14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14272108,2019-04-19,747,1916,2156.0,160.0,10.0,2206.0,2400.0,6.0,2100,2104.0,4.0,0.0,0.0,190.0,192.0,176.0,1182.0,2019,4,19,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6164840,2019-11-09,677,1815,1806.0,-9.0,11.0,1817.0,1948.0,4.0,1805,1757.0,-8.0,0.0,0.0,145.0,141.0,129.0,972.0,2019,11,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9767090,2019-01-24,689,2015,2342.0,207.0,8.0,2350.0,220.0,13.0,2120,2053.0,-27.0,0.0,0.0,189.0,163.0,140.0,1017.0,2019,1,24,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from scipy.signal import correlate
corr_matrix = df_sub.corr().abs()

In [ ]:
corr_matrix

,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,distance,year,month,day
crs_dep_time,1.000000,0.898152,0.094090,0.008026,0.874446,0.593102,0.010420,0.003603,0.003633,0.001355,0.007536,0.000888,0.005086,0.002690,0.003861,0.005281,0.000994,0.005376,0.004045
dep_time,0.898152,1.000000,0.118248,0.066919,0.975557,0.693895,0.010673,0.005677,0.000474,0.001481,0.006647,0.002692,0.004887,0.002968,0.004053,0.004968,0.002369,0.002863,0.000288
dep_delay,0.094090,0.118248,1.000000,0.069054,0.111617,0.035919,0.000212,0.005690,0.007540,0.000565,0.005196,0.000190,0.002136,0.003194,0.002718,0.002603,0.014788,0.000217,0.003209
taxi_out,0.008026,0.066919,0.069054,1.000000,0.097843,0.090049,0.003163,0.000725,0.002840,0.003949,0.005090,0.006811,0.001837,0.000303,0.001353,0.002815,0.010090,0.005398,0.010265
wheels_off,0.874446,0.975557,0.111617,0.097843,1.000000,0.713230,0.010596,0.005438,0.000680,0.000458,0.007872,0.003418,0.004334,0.002386,0.003335,0.004482,0.002803,0.001682,0.001863
wheels_on,0.593102,0.693895,0.035919,0.090049,0.713230,1.000000,0.005740,0.009959,0.003258,0.002387,0.008501,0.000150,0.004814,0.002885,0.003519,0.004229,0.012942,0.001191,0.003031
taxi_in,0.010420,0.010673,0.000212,0.003163,0.010596,0.005740,1.000000,0.006381,0.062107,0.098026,0.159139,0.032968,0.109784,0.192520,0.106838,0.076142,0.000593,0.006155,0.010541
crs_arr_time,0.003603,0.005677,0.005690,0.000725,0.005438,0.009959,0.006381,1.000000,0.783665,0.090379,0.018226,0.008047,0.015675,0.011394,0.009597,0.010862,0.010390,0.002073,0.001790
arr_time,0.003633,0.000474,0.007540,0.002840,0.000680,0.003258,0.062107,0.783665,1.000000,0.032044,0.331377,0.013657,0.019887,0.090264,0.073507,0.016165,0.001298,0.002594,0.005085
arr_delay,0.001355,0.001481,0.000565,0.003949,0.000458,0.002387,0.098026,0.090379,0.032044,1.000000,0.014023,0.005272,0.012933,0.045951,0.008294,0.013649,0.004433,0.000565,0.004395


In [ ]:
#Correlation with output variable
cor_target = abs(corr_matrix["arr_delay"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.6]
relevant_features

arr_delay    1.0
Name: arr_delay, dtype: float64

In [ ]:
# cumulative variance explained graph 
# do this with df_dummies 